# Actividad 1:
# Comparación de técnicas avanzadas para predicción de ingresos

## Objetivo
Objetivo: Aplicar y comparar modelos avanzados de regresión y clasificación sobre un mismo problema, evaluando su rendimiento y adecuación al contexto, utilizando un enfoque práctico e interpretativo.

**Dataset utilizado:**  
**`Adult Income`**

---

### Estructura del Notebook:
1. Metodología.
2. Importación de librerias a utilizar.
3. Definicion de funciones.
4. Uso de funciones y resultados.
5. Análisis de los resultados y reflexiones finales.

---

### Versión de librerías usadas:
- Pandas:           2.1.3
- NumPy:            1.26.2
- Matplotlib:       3.8.2
- Seaborn:          0.13.0
- Scikit-learn:     1.7.0
- XGBoost:          2.1.3
- Statsmodels:      0.14.0

---

## 1. Metodología

### Flujo de trabajo

1. **Carga y exploración de datos:**
   - Se utilizó el dataset **Adult Income** desde `fetch_openml`.
   - Se exploraron dimensiones, tipos de variables, y se definió la variable objetivo (`>50K` = 1, `<=50K` = 0).

2. **Preprocesamiento:**
   - Separación de variables numéricas y categóricas.
   - Codificación de variables categóricas mediante `OneHotEncoder`.
   - Estandarización de variables numéricas con `StandardScaler`.
   - División estratificada del dataset en conjuntos de entrenamiento (70%) y prueba (30%) para asegurar representatividad de clases.

3. **Entrenamiento de modelos:**
   - Se entrenaron y evaluaron cinco modelos distintos:
     - **ElasticNet** (modelo lineal penalizado).
     - **Regresiones Cuantílicas** con `statsmodels` en tres cuantiles: 0.1, 0.5 y 0.9.
     - **Random Forest Classifier**.
     - **XGBoost Classifier**.
   - Todos los modelos fueron integrados en pipelines (`Pipeline`) con los mismos pasos de preprocesamiento para asegurar una evaluación justa.
   - La regresión cuantilica se implementó fuera de `sklearn`, por lo que se preprocesaron manualmente los datos y se evaluaron los resultados cuantiles como clasificación (≥ 0.5).

4. **Evaluación y análisis:**
   - Métricas empleadas:
     - **Accuracy:** porcentaje de predicciones correctas.
     - **AUC (Area Under Curve):** mide la capacidad del modelo para distinguir entre clases.
     - **Pinball Loss:** métrica específica para regresión cuantilica que penaliza predicciones según el cuantil definido.
   - Se evaluaron las matrices de confusión para cada modelo.
   - Se extrajo y comparó la **importancia de variables** más influyentes en cada modelo.

5. **Visualización de resultados:**
   - Tabla resumen con métricas comparativas.
   - Gráficos de barras comparativos para Accuracy, AUC y curvas ROC.
   - Matrices de confusión por grupo de modelos.
   - Gráficos de las cinco variables más importantes por modelo.

---

# 2. Importacion de librerias necesarias

--- 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, confusion_matrix, roc_auc_score,
    mean_pinball_loss, roc_curve, auc
)
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import statsmodels.api as sm

# Configuración visual y de entorno
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')
pd.set_option('display.precision', 4)
pd.set_option('display.max_columns', None)

# 3. Definición de funciones

> **Nota:** Para mejor comprensión de las funciones y su utilidad, esta sección se divide en bloques, en donde cada uno responde a una parte diferente de la metodología de trabajo. 

---

**Bloque 1:** Funciones de preprocesamiento de datos.

- **`cargar_datos()`** 
Carga y prepara el dataset Adult Income, separando variables numéricas y categóricas. Tambien reemplaza los valores faltantes (que son aproximadamente 7.4%) por el valor mas frecuente (moda).

- **`preprocesador()`** 
Crea un transformador que escala variables numéricas y codifica variables categóricas.

In [ ]:
def cargar_datos():
    """
    Carga el dataset 'Adult Income' desde OpenML, transforma la variable objetivo 
    a formato binario y separa variables predictoras categóricas y numéricas.
    Reemplaza '?' por NaN para poder imputar valores faltantes luego.

    Returns:
        X (pd.DataFrame): DataFrame con las variables independientes (features).
        y (pd.Series): Serie con la variable objetivo binaria (1 si ingreso >50K, 0 si no).
        cat_cols (list): Lista con los nombres de columnas categóricas.
        num_cols (list): Lista con los nombres de columnas numéricas.
    """
    print("[INFO] Cargando datos...")
    data = fetch_openml("adult", version=2, as_frame=True)
    df = data.frame.copy()
    df['target'] = df['class'].apply(lambda x: 1 if x == '>50K' else 0)
    df.drop(columns=['class'], inplace=True)

    # Reemplazar '?' por np.nan para detectar valores faltantes
    df.replace('?', np.nan, inplace=True)

    X = df.drop(columns=['target'])
    y = df['target']

    cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
    num_cols = X.select_dtypes(include='number').columns.tolist()

    return X, y, cat_cols, num_cols


def preprocesador(cat_cols, num_cols):
    """
    Crea un transformador de columnas que:
    - Imputa valores faltantes en categóricas con la categoría más frecuente
    - Aplica escalado a variables numéricas
    - Codifica variables categóricas con OneHotEncoder

    Args:
        cat_cols (list): Lista con nombres de variables categóricas.
        num_cols (list): Lista con nombres de variables numéricas.

    Returns:
        ColumnTransformer: Objeto que transforma numéricas con StandardScaler y categóricas con imputación + OneHotEncoder.
    """

    num_pipeline = Pipeline([
        ('scaler', StandardScaler())
    ])

    cat_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    return ColumnTransformer([
        ('num', num_pipeline, num_cols),
        ('cat', cat_pipeline, cat_cols)
    ])


**Bloque 2:** Entrenamiento de modelos, evaluación y cálculo de importancia de variables.

- **`evaluar_modelo()`** 
Calcula métricas y matriz de confusión, guarda resultados e imprime resumen.

- **`entrenar_modelos()`** 
Entrena varios modelos y devuelve sus resultados, matrices y objetos.

- **`calcular_importancias()`** 
Extrae y muestra las variables más importantes de cada modelo entrenado.

In [ ]:
def evaluar_modelo(y_true, y_pred, nombre, results, conf_matrices):
    """
    Calcula métricas de evaluación (accuracy y AUC) para un modelo y actualiza 
    los diccionarios de resultados y matrices de confusión.

    Args:
        y_true (array-like): Valores reales de la variable objetivo.
        y_pred (array-like): Predicciones del modelo (clase binaria).
        nombre (str): Nombre del modelo para registrar en los diccionarios.
        results (dict): Diccionario donde se almacenan accuracy y AUC por modelo.
        conf_matrices (dict): Diccionario donde se almacenan las matrices de confusión por modelo.

    Returns:
        tuple: Accuracy y AUC del modelo evaluado.
    """
    acc = accuracy_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    results[nombre] = (acc, auc)
    conf_matrices[nombre] = cm
    print(f"\n[{nombre}] Accuracy: {acc:.4f}, AUC: {auc:.4f}")
    return acc, auc

def entrenar_modelos(X_train, y_train, X_test, y_test, preprocessor, cat_cols, num_cols):
    """
    Entrena y evalúa varios modelos de clasificación y regresión cuantilica sobre el dataset de Adult Income.

    Modelos incluidos:
        - ElasticNet (lineal penalizado)
        - Quantile Regression con statsmodels (q = 0.1, 0.5, 0.9)
        - Random Forest
        - XGBoost

    Args:
        X_train (pd.DataFrame): Conjunto de entrenamiento (features).
        y_train (pd.Series): Etiquetas del conjunto de entrenamiento.
        X_test (pd.DataFrame): Conjunto de prueba (features).
        y_test (pd.Series): Etiquetas del conjunto de prueba.
        preprocessor (ColumnTransformer): Objeto de preprocesamiento compartido por los modelos.
        cat_cols (list): Lista de columnas categóricas.
        num_cols (list): Lista de columnas numéricas.

    Returns:
        tuple:
            - results (dict): Métricas de accuracy y AUC por modelo.
            - conf_matrices (dict): Matrices de confusión por modelo.
            - model_store (dict): Modelos entrenados (pipeline o regresor statsmodels).
            - all_feature_names (list): Lista con todos los nombres de variables después del preprocesamiento.
    """
    results, conf_matrices, model_store = {}, {}, {}

    # ElasticNet
    print("[INFO] Entrenando ElasticNet...")
    elastic_pipe = Pipeline([
        ('preproc', preprocessor),
        ('model', ElasticNet(alpha=0.1, l1_ratio=0.5))
    ])
    elastic_pipe.fit(X_train, y_train)
    y_pred_en = elastic_pipe.predict(X_test)
    y_pred_en_class = (y_pred_en >= 0.5).astype(int)
    evaluar_modelo(y_test, y_pred_en_class, "ElasticNet", results, conf_matrices)
    model_store["ElasticNet"] = elastic_pipe

    # Quantile Regression
    print("[INFO] Entrenando Regresión Cuantílica con statsmodels...")
    preprocessor_qr = preprocesador(cat_cols, num_cols)
    X_train_trans = preprocessor_qr.fit_transform(X_train)
    X_test_trans = preprocessor_qr.transform(X_test)

    cat_features = preprocessor_qr.named_transformers_['cat'].get_feature_names_out(cat_cols)
    all_feature_names = np.concatenate([num_cols, cat_features])
    X_train_df = pd.DataFrame(X_train_trans.toarray() if hasattr(X_train_trans, 'toarray') else X_train_trans,
                              columns=all_feature_names, index=X_train.index)
    X_test_df = pd.DataFrame(X_test_trans.toarray() if hasattr(X_test_trans, 'toarray') else X_test_trans,
                             columns=all_feature_names, index=X_test.index)

    X_train_const = sm.add_constant(X_train_df)
    X_test_const = sm.add_constant(X_test_df)

    for q in [0.1, 0.5, 0.9]:
        print(f"  > Entrenando QuantileReg q={q} con statsmodels...")
        qr_model = sm.QuantReg(y_train, X_train_const)
        res = qr_model.fit(q=q)
        y_pred_q = res.predict(X_test_const)
        y_pred_q_class = (y_pred_q >= 0.5).astype(int)
        nombre = f"Quantile q={q}"
        evaluar_modelo(y_test, y_pred_q_class, nombre, results, conf_matrices)
        pinball = mean_pinball_loss(y_test, y_pred_q, alpha=q)
        print(f"[{nombre}] Pinball Loss: {pinball:.4f}")
        model_store[nombre] = res

    # Random Forest
    print("[INFO] Entrenando Random Forest...")
    rf_pipe = Pipeline([
        ('preproc', preprocessor),
        ('model', RandomForestClassifier(n_estimators=100, random_state=42))
    ])
    rf_pipe.fit(X_train, y_train)
    y_pred_rf = rf_pipe.predict(X_test)
    evaluar_modelo(y_test, y_pred_rf, "Random Forest", results, conf_matrices)
    model_store['Random Forest'] = rf_pipe

    # XGBoost
    print("[INFO] Entrenando XGBoost...")
    xgb_pipe = Pipeline([
        ('preproc', preprocessor),
        ('model', XGBClassifier(eval_metric='logloss', random_state=42))
    ])
    xgb_pipe.fit(X_train, y_train)
    y_pred_xgb = xgb_pipe.predict(X_test)
    evaluar_modelo(y_test, y_pred_xgb, "XGBoost", results, conf_matrices)
    model_store['XGBoost'] = xgb_pipe

    return results, conf_matrices, model_store, all_feature_names

def calcular_importancias(model_store, all_feature_names):
    """
    Calcula e imprime la importancia de las variables para cada modelo.

    Para modelos lineales (ElasticNet y Quantile Regression) se usa el valor absoluto de los coeficientes.
    Para modelos basados en árboles (Random Forest y XGBoost) se usa `feature_importances_`.

    Args:
        model_store (dict): Diccionario con los modelos entrenados.
        all_feature_names (list): Lista de nombres de todas las variables transformadas.

    Returns:
        dict: Diccionario con las 5 variables más importantes por modelo.
    """
    print("\n[IMPORTANCIA DE VARIABLES]")
    importancias = {}
    for modelo in ['ElasticNet', 'Random Forest', 'XGBoost']:
        pipe = model_store[modelo]
        if hasattr(pipe.named_steps['model'], 'coef_'):
            coefs = np.abs(pipe.named_steps['model'].coef_)
        else:
            coefs = pipe.named_steps['model'].feature_importances_
        top_idx = np.argsort(coefs)[-5:][::-1]
        importancias[modelo] = [(all_feature_names[i], coefs[i]) for i in top_idx]

    for q in [0.1, 0.5, 0.9]:
        modelo = f"Quantile q={q}"
        res = model_store[modelo]
        coefs = np.abs(res.params.values[1:])  # sin intercepto
        top_idx = np.argsort(coefs)[-5:][::-1]
        importancias[modelo] = [(all_feature_names[i], coefs[i]) for i in top_idx]

    print("\nTop 1 variable por modelo:")
    for modelo, top_feats in importancias.items():
        print(f"{modelo:<15}: {top_feats[0][0]} ({top_feats[0][1]:.4f})")
    return importancias

**Bloque 3:** Resultados y visualizaciones.

- **`mostrar_resultados()`** 
Muestra y retorna un DataFrame con las métricas de desempeño de todos los modelos.

- **`graficar_importancias()`** 
Genera gráficos de barras con las importancias de las variables para cada modelo.

- **`graficar_metricas()`** 
Visualiza comparaciones de accuracy y AUC de los modelos con gráficos de barras, además de una comparación de las curvas ROC.

- **`graficar_confusiones()`** 
Muestra las matrices de confusión de los modelos en forma de mapas de calor.

In [ ]:
def mostrar_resultados(results):
    """
    Muestra una tabla con las métricas de rendimiento (Accuracy y AUC) para cada modelo.

    Args:
        results (dict): Diccionario con los nombres de los modelos como llaves y tuplas (accuracy, AUC) como valores.

    Returns:
        pd.DataFrame: DataFrame con los resultados por modelo.
    """
    print("\n[RESULTADOS FINALES]")
    df = pd.DataFrame(results).T
    df.columns = ['Accuracy', 'AUC']
    print(df)
    return df

def graficar_importancias(importancias):
    """
    Genera gráficos de la importancia de variables para ElasticNet, Random Forest, XGBoost 
    y para los modelos de regresión cuantilica (q=0.1, 0.5, 0.9).

    Args:
        importancias (dict): Diccionario con los nombres de los modelos como llaves 
                             y listas de tuplas (nombre_variable, importancia) como valores.
    """
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))  

    # Definir colores para cada modelo
    colores = ['green', 'orange', 'purple']

    # Primera fila: ElasticNet, Random Forest, XGBoost
    for ax, modelo, color in zip(axes[0], ['ElasticNet', 'Random Forest', 'XGBoost'], colores):
        nombres, pesos = zip(*importancias[modelo])
        sns.barplot(x=nombres, y=pesos, ax=ax, color=color)
        ax.set_title(f"{modelo} - Top 5 features")
        ax.tick_params(axis='x', rotation=45)

    # Segunda fila: Quantile q=0.1, q=0.5, q=0.9
    for ax, modelo in zip(axes[1], ['Quantile q=0.1', 'Quantile q=0.5', 'Quantile q=0.9']):
        nombres, pesos = zip(*importancias[modelo])
        sns.barplot(x=nombres, y=pesos, ax=ax, color='DarkBlue')
        ax.set_title(f"{modelo} - Top 5 features")
        ax.tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.savefig('importancias_variables.png', dpi=300)
    plt.show()

def graficar_metricas(df, model_store, X_test, y_test, X_test_qr):
    """
    Genera una figura con tres subgráficos:
    - Gráfico de barras comparando Accuracy entre modelos.
    - Gráfico de barras comparando AUC entre modelos.
    - Curvas ROC para todos los modelos, incluyendo regresión cuantilica.

    Args:
        df (pd.DataFrame): DataFrame con métricas de evaluación (Accuracy, AUC).
        model_store (dict): Diccionario con los modelos entrenados.
        X_test (pd.DataFrame): Conjunto de test sin preprocesar.
        y_test (pd.Series): Etiquetas verdaderas del conjunto de test.
        X_test_qr (np.ndarray): Conjunto de test transformado para regresión cuantilica (ya con constante).
    """
    fig, axes = plt.subplots(1, 3, figsize=(21, 6))

    # Gráfico 1: Accuracy
    sns.barplot(x=df.index, y='Accuracy', data=df.reset_index(), ax=axes[0], color='Red')
    axes[0].set_title('Comparación de Accuracy')
    axes[0].tick_params(axis='x', rotation=45)
    axes[0].grid(axis='y')

    # Gráfico 2: AUC
    sns.barplot(x=df.index, y='AUC', data=df.reset_index(), ax=axes[1], color='Red')
    axes[1].set_title('Comparación de AUC')
    axes[1].tick_params(axis='x', rotation=45)
    axes[1].grid(axis='y')

    # Gráfico 3: Curvas ROC

    for modelo in ['ElasticNet', 'Random Forest', 'XGBoost']:
        pipe = model_store[modelo]
        y_scores = pipe.predict_proba(X_test)[:, 1] if hasattr(pipe.named_steps['model'], 'predict_proba') else pipe.predict(X_test)
        fpr, tpr, _ = roc_curve(y_test, y_scores)
        roc_auc = auc(fpr, tpr)
        axes[2].plot(fpr, tpr, lw=2, label=f"{modelo} (AUC = {roc_auc:.2f})")

    # Cuantil
    for q in [0.1, 0.5, 0.9]:
        modelo = f"Quantile q={q}"
        res = model_store[modelo]
        y_scores = res.predict(X_test_qr)
        fpr, tpr, _ = roc_curve(y_test, y_scores)
        roc_auc = auc(fpr, tpr)
        axes[2].plot(fpr, tpr, linestyle='--', lw=1.5, label=f"{modelo} (AUC = {roc_auc:.2f})")

    axes[2].plot([0, 1], [0, 1], 'k--', lw=1)
    axes[2].set_title('Curvas ROC')
    axes[2].set_xlabel('False Positive Rate')
    axes[2].set_ylabel('True Positive Rate')
    axes[2].legend(loc='lower right')
    axes[2].grid(True)

    plt.tight_layout()
    plt.savefig('metricas_modelos.png', dpi=300)
    plt.show()

def graficar_confusiones(conf_matrices):
    """
    Genera matrices de confusión para ElasticNet, Random Forest, XGBoost 
    y para los modelos de regresión cuantilica (q=0.1, 0.5, 0.9).

    Args:
        conf_matrices (dict): Diccionario con los nombres de los modelos como llaves 
                              y matrices de confusión (2x2 np.ndarray) como valores.
                              
    """
    fig, axes = plt.subplots(2, 3, figsize=(18, 10)) 

    # Primera fila: ElasticNet, Random Forest, XGBoost con cmaps diferentes
    cmaps = ['Greens', 'Oranges', 'Purples']
    modelos = ['ElasticNet', 'Random Forest', 'XGBoost']

    for ax, modelo, cmap in zip(axes[0], modelos, cmaps):
        sns.heatmap(conf_matrices[modelo], annot=True, fmt='d', cmap=cmap, ax=ax)
        ax.set_title(f'Matriz de Confusión - {modelo}')

    # Segunda fila: regresiones cuantílicas con cmap 'Blues'
    for ax, modelo in zip(axes[1], ['Quantile q=0.1', 'Quantile q=0.5', 'Quantile q=0.9']):
        sns.heatmap(conf_matrices[modelo], annot=True, fmt='d', cmap='Blues', ax=ax)
        ax.set_title(f'Matriz de Confusión - {modelo}')

    plt.tight_layout()
    plt.savefig('matrices_confusion.png', dpi=300)
    plt.show()

**Bloque 4:** Función de ejecución.

- **`main()`**
Utiliza todas las funciones definidas anteriormente para obtener datos de los modelos mediante Elastic Net, Regresión Cuantílica (q = 0.1, 0.5 y 0.9) y XGBoost

In [ ]:
def main():
    """
    Ejecuta el flujo completo de carga, preprocesamiento, entrenamiento, evaluación y visualización 
    de modelos para el dataset Adult Income.

    Pasos incluidos:
    - Carga y separación de datos en variables predictoras y objetivo.
    - Preprocesamiento (escalado y codificación).
    - División en conjuntos de entrenamiento y prueba.
    - Entrenamiento de múltiples modelos (ElasticNet, regresión cuantilica, Random Forest, XGBoost).
    - Evaluación de métricas de rendimiento y cálculo de importancia de variables.
    - Presentación de resultados numéricos y visuales (gráficos de importancia, métricas y matrices de confusión).
    """
    X, y, cat_cols, num_cols = cargar_datos()
    preproc = preprocesador(cat_cols, num_cols)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

    results, conf_matrices, model_store, all_feature_names = entrenar_modelos(
        X_train, y_train, X_test, y_test, preproc, cat_cols, num_cols
    )
    importancias = calcular_importancias(model_store, all_feature_names)
    df_resultados = mostrar_resultados(results)

    # Preparar X_test_qr (transformado + constante) para curvas ROC de regresión cuantilica
    preprocessor_qr = preprocesador(cat_cols, num_cols)
    X_test_trans = preprocessor_qr.fit(X_train).transform(X_test)
    if hasattr(X_test_trans, 'toarray'):
        X_test_trans = X_test_trans.toarray()
    X_test_qr = sm.add_constant(X_test_trans)

    graficar_importancias(importancias)
    graficar_metricas(df_resultados, model_store, X_test, y_test, X_test_qr)  # ahora con curva ROC integrada
    graficar_confusiones(conf_matrices)

# 4. Visualización de resultados

Se muestran los resultados obtenidos a partir de la ejecución de la funcion **main()**.

---

In [ ]:
if __name__ == "__main__":
    main()

## Análisis Comparativo de Modelos de Clasificación - Adult Income Dataset

---

### Justificaciones

**Limpieza de datos:** El dataset mostró tener aproximadamente 3600 NaN, que representan alrededor del 7.4% del conjunto total. Visto que es un % significativo, se decidió imputar los datos con el mas frecuente (moda), debido a que eliminar esa cantidad de datos podría llevar a un sesgo en los resultados finales.

**Uso de métodos de optimización**: Para efectos prácticos, debido a que la finalidad de este trabajo es poner a prueba distintos modelos como Elastic Net o XGBoost, para no hacer un código tan extenso y evitar que la ejecución tome mucho tiempo, se omitió la realizacion de métodos de optimización como grid search o random search para buscar los mejores hiperparámetros para cada modelos.

---

>**Nota:** 
>
> Este análisis incluye la evaluación de varios modelos de aprendizaje automático sobre el dataset Adult Income, cuyo objetivo es clasificar si una persona gana más de $50.000 anuales.
>
> Algunos de los modelos utilizados, como ElasticNet (regresión lineal) y Regresión Cuantilica, no están diseñados específicamente para clasificación binaria. Su inclusión en este trabajo tiene un propósito comparativo y exploratorio, para analizar cómo se comportan en un contexto fuera de su aplicación ideal.
>
> Por ello, métricas típicas de regresión como RMSE no se aplican ni se reportan aquí. En cambio, para evaluar su desempeño en esta tarea usamos Accuracy, AUC y Pinball Loss (específica para regresión cuantilica). Esto implica que las comparaciones deben interpretarse considerando estas diferencias metodológicas.

---

### 1. ¿Cuál modelo rinde mejor en qué contexto?

- **XGBoost** obtuvo el mejor rendimiento general con una **accuracy de 87.76%** y **AUC de 0.805**, siendo el modelo más robusto para predecir si una persona gana más de \$50.000.
- **Random Forest** mostró un desempeño sólido (**accuracy 85.63%**, **AUC 0.775**), útil si se busca un equilibrio entre interpretabilidad (mediante importancia de variables o herramientas como SHAP) y precisión.
- **ElasticNet** logró una accuracy de 77.30% pero con una baja AUC (0.53), indicando poca capacidad discriminativa pese a una precisión moderada.
- **Regresiones cuantilicas** mostraron resultados variables:
  - Cuantil **q=0.5 (mediana)** tuvo desempeño razonable (accuracy 78.69%, AUC 0.60), con un valor de Pinball Loss que mide el error específico para ese cuantíl, indicando qué tan bien el modelo estima esa parte de la distribución del ingreso.
  - Cuantiles extremos (q=0.1 y q=0.9) mostraron menor estabilidad: q=0.1 predijo aleatoriamente (AUC ≈ 0.5) y un Pinball Loss bajo debido a la naturaleza del cuantíl bajo, mientras que q=0.9, aunque logró una alta AUC, tuvo baja precisión y un Pinball Loss intermedio, posiblemente por sobreajuste en un subconjunto minoritario.

### **Resumen general:**
| Criterio                      | Mejor modelo        |
|------------------------------|---------------------|
| Mayor precisión y AUC        | XGBoost             |
| Interpretabilidad balanceada | Random Forest       |
| Alternativas lineales        | Menor desempeño     |

---

### 2. ¿Qué variable tuvo más impacto en cada modelo?

Los resultados muestran que las variables con mayor impacto varían según el modelo y el cuantíl analizado, reflejando diferentes perspectivas y sensibilidad a las características del dataset:

| Modelo          | Variable más importante                |Peso (App)|
|-----------------|----------------------------------------|----------|
| ElasticNet      | marital-status_Married-civ-spouse      | 0.1192   |
| Quantile q=0.1  | capital-gain                           | 0.0000   |
| Quantile q=0.5  | education_Prof-school                  | 0.4172   |
| Quantile q=0.9  | native-country_Hungary                 | 0.9652   |
| Random Forest   | fnlwgt                                 | 0.1689   |
| XGBoost         | marital-status_Married-civ-spouse      | 0.3760   |

---

- **Modelos lineales y regularizados (ElasticNet)** coinciden en que el **estado civil**, específicamente estar casado ("married-civ-spouse"), es un predictor clave para determinar ingresos altos. Esto indica que esta variable tiene un efecto lineal fuerte y consistente en la probabilidad de ganar más de \$50K.

- En la **regresión cuantílica**, la importancia de las variables cambia según el cuantíl:
  - En el **cuantíl bajo (q=0.1)**, la variable "capital-gain" tiene peso nulo, sugiriendo que para los ingresos más bajos no aporta predictivamente.
  - En el **cuantíl mediano (q=0.5)**, "education_Prof-school" (nivel educativo avanzado) cobra relevancia, indicando que a niveles medios de ingreso la educación tiene un impacto significativo.
  - En el **cuantíl alto (q=0.9)**, "native-country_Hungary" destaca con un peso muy alto, lo que podría reflejar que para los ingresos más altos el país de origen o factores demográficos específicos juegan un papel importante, aunque podría estar influenciado por menor cantidad de casos o sesgos.

- Los **modelos basados en árboles (Random Forest y XGBoost)** resaltan variables distintas:
  - **Random Forest** destaca "fnlwgt" (peso final), una variable relacionada con la ponderación del encuestado en la muestra, que puede capturar características complejas del dataset.
  - **XGBoost** vuelve a enfatizar "marital-status_Married-civ-spouse", mostrando la importancia continua de esta variable también en modelos no lineales y más complejos.

En resumen, el **estado civil** y el **nivel educativo** aparecen como predictores recurrentes y clave en la mayoría de los modelos, mientras que otras variables, como el país de origen o características específicas de la muestra, influyen dependiendo del método y la parte de la distribución de ingresos que se modele.

---

### 3. ¿Qué modelo recomendarías implementar?

Recomendaría implementar **XGBoost**, ya que ofrece:
- La mejor precisión
- Capacidad para modelar relaciones no lineales complejas
- Buen manejo de outliers y features categóricos
- Compatibilidad con interpretabilidad moderna mediante técnicas como SHAP o PDP

Alternativamente, si se necesita mayor **transparencia directa en las decisiones**, **Random Forest** es una opción sólida, aunque con leve pérdida de precisión.

---

### 4. Reflexión: ¿Por qué ElasticNet y regresión cuantilica no son preferibles aquí?

### ElasticNet:
- Es un modelo **lineal** que no captura bien las interacciones complejas entre variables presentes en este problema.
- Aunque su accuracy es razonable, su **AUC de 0.53** indica que no distingue adecuadamente entre clases.

### Regresión Cuantílica:
- Está diseñada para predecir **distribuciones de variables continuas**, no para clasificación binaria.
- En este contexto, genera probabilidades poco calibradas.
- Además, es **computacionalmente costosa** cuando hay muchas variables categóricas codificadas.

---

## Conclusiones finales

Este análisis comparativo confirma que:

- **Modelos basados en árboles**, como XGBoost y Random Forest, son ideales para tareas de clasificación en datasets como *Adult Income*.
- La **elección del modelo** debe equilibrar precisión, interpretabilidad y recursos computacionales.
- La exploración de variables clave permite **interpretar el comportamiento del modelo** y su posible sesgo en decisiones sensibles como predicción de ingresos.

---